In [8]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama3-70b-8192")

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002798540D5B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002798540DBE0>, model_name='llama3-70b-8192', groq_api_key=SecretStr('**********'))

In [10]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
groq_api_key=os.getenv("GROQ_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\yash\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\yash\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import FAISS

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [14]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
import bs4
loader = WebBaseLoader(
    web_paths=("https://www.flipkart.com/search?q=mobiles&sid=tyy%2C4io&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_2_7_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_2_7_na_na_na&as-pos=2&as-type=RECENT&suggestionId=mobiles%7CMobiles&requestId=6b019d01-4d80-43a5-ba42-9f39c182b117&as-searchtext=mobiles",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("DOjaWF gdgoEp","DOjaWF gdgoEp","DOjaWF gdgoEp")
        )
    ),
)

docs=loader.load()
docs

[Document(metadata={'source': 'https://www.flipkart.com/search?q=mobiles&sid=tyy%2C4io&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_2_7_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_2_7_na_na_na&as-pos=2&as-type=RECENT&suggestionId=mobiles%7CMobiles&requestId=6b019d01-4d80-43a5-ba42-9f39c182b117&as-searchtext=mobiles'}, page_content='HomeMobiles & AccessoriesMobilesShowing 1 â€“ 24 of 10,063 results for "mobiles"Sort ByRelevancePopularityPrice -- Low to HighPrice -- High to LowNewest FirstAdd to Comparevivo V40 5G (Ganges Blue, 512 GB)4.599 RatingsÂ\xa0&Â\xa015 Reviews12 GB RAM | 512 GB ROM17.22 cm (6.78 inch) Full HD+ Display50MP + 50MP | 50MP Front Camera5500 mAh BatterySnapdragon 7 Gen 3 Processor1 Year Warranty on Handset and 6 Months Warranty on Accessoriesâ‚¹41,999â‚¹48,99914% offFree deliverySave extra with combo offersUpto â‚¹34,800 Off on ExchangeAdd to CompareApple iPhone 15 (Blue, 128 GB)4.648,096 RatingsÂ\xa0&Â\xa02,531 Reviews128 GB ROM15.49 cm (6.1 inc

In [52]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits=text_splitter.split_documents(docs)
vectorstore=FAISS.from_documents(documents=splits,embedding=embeddings)
retriever=vectorstore.as_retriever() #Interface for vectordb
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002798B936360>)

In [18]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002798B827950>)

In [20]:
# formatted_docs = [{"page_content": doc.page_content, "metadata": doc.metadata} for doc in docs]

In [54]:
# formatted_docs 

In [36]:
# type(formatted_docs)

list

In [56]:
# type(formatted_docs[0])

In [58]:
# formatted_docs[0]['page_content']

In [60]:
# for i in formatted_docs:
#     print(i['page_content'])

In [62]:
# text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
# splits=text_splitter.split_documents(formatted_docs[0]['page_content'])
# vectorstore=FAISS.from_documents(documents=splits,embedding=embeddings)
# retriever=vectorstore.as_retriever() #Interface for vectordb
# retriever

In [68]:
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import initialize_agent,AgentType

retriever_tool=create_retriever_tool(retriever,"flipchat","flipkart mobile search")

retriever_tool
tools=[retriever_tool]
search_agent=initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,handling_parsing_errors=True)

C:\ProgramData\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [76]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

C:\ProgramData\anaconda3\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [89]:
from langchain.agents import AgentExecutor, create_structured_chat_agent




